<a href="https://colab.research.google.com/github/vagenas/llama_index/blob/add-docling-reader/docs/docs/examples/data_connectors/DoclingPDFReaderDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Docling PDF Reader

## Overview

[Docling](https://github.com/DS4SD/docling) extracts PDF content into a rich representation (incl. layout, tables etc.) which it can export to Markdown or JSON.

In the sections below, we show how to use `DoclingPDFReader` to bring PDF files into your RAG pipeline with ease and speed — while also leveraging Docling's rich format for advanced, document-native grounding.

## Setup

In [ ]:
%pip install -q llama-index-readers-docling llama-index-node-parser-docling llama-index-embeddings-huggingface llama-index-llms-huggingface-api llama-index-vector-stores-milvus python-dotenv rich

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import rich
from rich.pretty import pprint
from warnings import filterwarnings
import os
from tempfile import mkdtemp
from pathlib import Path
from dotenv import load_dotenv

load_dotenv()
filterwarnings(action="ignore", category=UserWarning, module="pydantic")
filterwarnings(action="ignore", category=FutureWarning, module="easyocr")
os.environ["TOKENIZERS_PARALLELISM"] = "false"


def get_env_var(key, default=None):
    try:
        from google.colab import userdata

        try:
            return userdata.get(key)
        except userdata.SecretNotFoundError:
            pass
    except ImportError:
        pass
    return os.environ.get(key, default)

In [ ]:
HF_TOKEN = get_env_var("HF_TOKEN")
MILVUS_URI = get_env_var("MILVUS_URI", str(Path(mkdtemp()) / "docling.db"))
FILE_PATH = "https://arxiv.org/pdf/2408.09869"  # Docling Technical Report
QUERY = "Which are the main AI models in Docling?"

## Using Markdown export

By default, `DoclingPDFReader` exports to Markdown. Basic usage looks like this:

In [ ]:
from llama_index.readers.docling import DoclingPDFReader

reader = DoclingPDFReader()
docs = reader.load_data(file_path=FILE_PATH)

Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

Let's inspect a doc snippet:

In [ ]:
md_snippet = docs[0].text[408:1200]
rich.print(f"...{md_snippet}...")

...
## Abstract

This technical report introduces Docling , an easy to use, self-contained, MITlicensed open-source package for PDF 
document conversion. It is powered by state-of-the-art specialized AI models for layout analysis (DocLayNet) and 
table structure recognition (TableFormer), and runs efficiently on commodity hardware in a small resource budget. 
The code interface allows for easy extensibility and addition of new features and models.

## 1 Introduction

Converting PDF documents back into a machine-processable format has been a major challenge for decades due to their
huge variability in formats, weak standardization and printing-optimized characteristic, which discards most 
structural features and metadata. With the advent of LLMs and popular application patterns such as re...

Next, we extract the chunks using a standard Markdown node parser:

In [ ]:
from llama_index.core.node_parser import MarkdownNodeParser

node_parser = MarkdownNodeParser()
nodes = node_parser.get_nodes_from_documents(documents=docs)

Let's preview an example chunk:

In [ ]:
pprint(nodes[7], max_length=2, max_string=250, max_depth=2)

TextNode(
│   id_='5e814bf9-0a22-4c42-a4a5-e704ef9cf385',
│   embedding=None,
│   metadata={
│   │   'dl_doc_hash': '556ad9e23b6d2245e36b3208758cf0c8a709382bb4c859eacfe8e73b14e635aa',
│   │   'Header_2': '3.1 PDF backends'
│   },
│   excluded_embed_metadata_keys=['dl_doc_hash'],
│   excluded_llm_metadata_keys=['dl_doc_hash'],
│   relationships={
│   │   <NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(...),
│   │   <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(...),
│   │   ... +1
│   },
│   text='3.1 PDF backends\n\nTwo basic requirements to process PDF documents in our pipeline are a) to retrieve all text content and their geometric coordinates on each page and b) to render the visual representation of each page as it would appear in a PDF vie'+1029,
│   mimetype='text/plain',
│   start_char_idx=5009,
│   end_char_idx=6288,
│   text_template='{metadata_str}\n\n{content}',
│   metadata_template='{key}: {value}',
│   metadata_seperator='\n'
)

We now put together a simple ingestion pipeline:

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
from llama_index.vector_stores.milvus import MilvusVectorStore
from llama_index.core import Settings, StorageContext, VectorStoreIndex

Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)
Settings.llm = HuggingFaceInferenceAPI(
    token=HF_TOKEN,
    model_name="mistralai/Mixtral-8x7B-Instruct-v0.1",
)


def ingest(nodes):
    vector_store = MilvusVectorStore(
        uri=MILVUS_URI,
        dim=len(Settings.embed_model.get_text_embedding("hi")),
        overwrite=True,
    )
    index = VectorStoreIndex(
        nodes=nodes,
        storage_context=StorageContext.from_defaults(
            vector_store=vector_store
        ),
        show_progress=True,
    )
    return index

With all pieces in place, we are now ready to ask questions on our document content:

In [ ]:
index = ingest(nodes)
query_res = index.as_query_engine().query(QUERY)
pprint(query_res, max_length=5, max_string=170, max_depth=4)

Generating embeddings:   0%|          | 0/33 [00:00<?, ?it/s]

Response(
│   response='\nThe main AI models in Docling are a layout analysis model, which is an accurate object-detector for page elements, and TableFormer, a state-of-the-art table structure re'+16,
│   source_nodes=[
│   │   NodeWithScore(
│   │   │   node=TextNode(
│   │   │   │   id_='f506b053-3d03-44d6-a396-e8d707cba476',
│   │   │   │   embedding=None,
│   │   │   │   metadata={...},
│   │   │   │   excluded_embed_metadata_keys=[...],
│   │   │   │   excluded_llm_metadata_keys=[...],
│   │   │   │   relationships={...},
│   │   │   │   text='3.2 AI models\n\nAs part of Docling, we initially release two highly capable AI models to the open-source community, which have been developed and published recently by our'+453,
│   │   │   │   mimetype='text/plain',
│   │   │   │   start_char_idx=6293,
│   │   │   │   end_char_idx=6916,
│   │   │   │   text_template='{metadata_str}\n\n{content}',
│   │   │   │   metadata_template='{key}: {value}',
│   │   │   │   metadata_seperator='\n'
│   │   │   ),
│   │   │   score=0.8273308277130127
│   │   ),
│   │   NodeWithScore(
│   │   │   node=TextNode(
│   │   │   │   id_='d9f3afb3-64fa-4cd9-b4db-d2442b53bbe8',
│   │   │   │   embedding=None,
│   │   │   │   metadata={...},
│   │   │   │   excluded_embed_metadata_keys=[...],
│   │   │   │   excluded_llm_metadata_keys=[...],
│   │   │   │   relationships={...},
│   │   │   │   text='5 Applications\n\nThanks to the high-quality, richly structured document conversion achieved by Docling, its output qualifies for numerous downstream applications. For exam'+1035,
│   │   │   │   mimetype='text/plain',
│   │   │   │   start_char_idx=14147,
│   │   │   │   end_char_idx=15352,
│   │   │   │   text_template='{metadata_str}\n\n{content}',
│   │   │   │   metadata_template='{key}: {value}',
│   │   │   │   metadata_seperator='\n'
│   │   │   ),
│   │   │   score=0.7876063585281372
│   │   )
│   ],
│   metadata={
│   │   'f506b053-3d03-44d6-a396-e8d707cba476': {
│   │   │   'dl_doc_hash': '556ad9e23b6d2245e36b3208758cf0c8a709382bb4c859eacfe8e73b14e635aa',
│   │   │   'Header_2': '3.2 AI models'
│   │   },
│   │   'd9f3afb3-64fa-4cd9-b4db-d2442b53bbe8': {
│   │   │   'dl_doc_hash': '556ad9e23b6d2245e36b3208758cf0c8a709382bb4c859eacfe8e73b14e635aa',
│   │   │   'Header_2': '5 Applications'
│   │   }
│   }
)

## Using Docling's native format

To leverage Docling's rich native format, this time we set `export_type` to `"json"`.

In [ ]:
reader = DoclingPDFReader(export_type="json")
docs = reader.load_data(file_path=FILE_PATH)

Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

Previewing a snippet of the doc content, we see it is indeed a JSON string:

In [ ]:
pprint(f"{docs[0].text[:50]}...")

'{"_name":"","type":"pdf-document","description":{"...'

In order to appropriately parse / chunk the above-generated Docling native format, we use a `DoclingNodeParser`:

In [ ]:
from llama_index.node_parser.docling import DoclingNodeParser

node_parser = DoclingNodeParser()
nodes = node_parser.get_nodes_from_documents(documents=docs)

Inspecting a node, we see it not only contains the text, but also various metadata:

In [ ]:
pprint(nodes[5], max_length=5, max_string=200, max_depth=2)

TextNode(
│   id_='b1ea55d7-98fc-4907-967d-5c8d628802bb',
│   embedding=None,
│   metadata={
│   │   'dl_doc_hash': '556ad9e23b6d2245e36b3208758cf0c8a709382bb4c859eacfe8e73b14e635aa',
│   │   'path': '#/main-text/21',
│   │   'heading': '2 Getting Started',
│   │   'page': 2,
│   │   'bbox': [...]
│   },
│   excluded_embed_metadata_keys=['dl_doc_hash', 'origin', 'path', 'page', 'bbox', ... +1],
│   excluded_llm_metadata_keys=['dl_doc_hash', 'origin', 'path', 'page', 'bbox', ... +1],
│   relationships={
│   │   <NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(...),
│   │   <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(...),
│   │   <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(...)
│   },
│   text='To use Docling, you can simply install the docling package from PyPI. Documentation and examples are available in our GitHub repository at github.com/DS4SD/docling. All required model assets 1 are dow'+121,
│   mimetype='text/plain',
│   start_char_idx=8531,
│   end_char_idx=8852,
│   text_template='{metadata_str}\n\n{content}',
│   metadata_template='{key}: {value}',
│   metadata_seperator='\n'
)

Let's now repeat the pipeline with these nodes.

As shown below, besides the response itself, we are also getting document-native grounding, incl. page number and bounding box information:

In [ ]:
index = ingest(nodes)
query_res = index.as_query_engine().query(QUERY)
pprint(query_res, max_length=5, max_string=170, max_depth=5)

Generating embeddings:   0%|          | 0/83 [00:00<?, ?it/s]

Response(
│   response=' The main AI models in Docling are a layout analysis model and TableFormer. The layout analysis model is an accurate object-detector for page elements, and TableFormer is'+54,
│   source_nodes=[
│   │   NodeWithScore(
│   │   │   node=TextNode(
│   │   │   │   id_='41ea2d41-b78e-4ff9-8328-81b33c42426d',
│   │   │   │   embedding=None,
│   │   │   │   metadata={
│   │   │   │   │   'dl_doc_hash': '556ad9e23b6d2245e36b3208758cf0c8a709382bb4c859eacfe8e73b14e635aa',
│   │   │   │   │   'path': '#/main-text/36',
│   │   │   │   │   'heading': '3.2 AI models',
│   │   │   │   │   'page': 3,
│   │   │   │   │   'bbox': [...]
│   │   │   │   },
│   │   │   │   excluded_embed_metadata_keys=['dl_doc_hash', 'origin', 'path', 'page', 'bbox', ... +1],
│   │   │   │   excluded_llm_metadata_keys=['dl_doc_hash', 'origin', 'path', 'page', 'bbox', ... +1],
│   │   │   │   relationships={
│   │   │   │   │   <NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(...),
│   │   │   │   │   <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(...),
│   │   │   │   │   <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(...)
│   │   │   │   },
│   │   │   │   text='As part of Docling, we initially release two highly capable AI models to the open-source community, which have been developed and published recently by our team. The firs'+438,
│   │   │   │   mimetype='text/plain',
│   │   │   │   start_char_idx=14945,
│   │   │   │   end_char_idx=15553,
│   │   │   │   text_template='{metadata_str}\n\n{content}',
│   │   │   │   metadata_template='{key}: {value}',
│   │   │   │   metadata_seperator='\n'
│   │   │   ),
│   │   │   score=0.8484227061271667
│   │   ),
│   │   NodeWithScore(
│   │   │   node=TextNode(
│   │   │   │   id_='5686aae4-1b36-46a4-8f82-e37d23f6d79f',
│   │   │   │   embedding=None,
│   │   │   │   metadata={
│   │   │   │   │   'dl_doc_hash': '556ad9e23b6d2245e36b3208758cf0c8a709382bb4c859eacfe8e73b14e635aa',
│   │   │   │   │   'path': '#/main-text/10',
│   │   │   │   │   'heading': '1 Introduction',
│   │   │   │   │   'page': 1,
│   │   │   │   │   'bbox': [...]
│   │   │   │   },
│   │   │   │   excluded_embed_metadata_keys=['dl_doc_hash', 'origin', 'path', 'page', 'bbox', ... +1],
│   │   │   │   excluded_llm_metadata_keys=['dl_doc_hash', 'origin', 'path', 'page', 'bbox', ... +1],
│   │   │   │   relationships={
│   │   │   │   │   <NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(...),
│   │   │   │   │   <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(...),
│   │   │   │   │   <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(...)
│   │   │   │   },
│   │   │   │   text='With Docling , we open-source a very capable and efficient document conversion tool which builds on the powerful, specialized AI models and datasets for layout analysis a'+318,
│   │   │   │   mimetype='text/plain',
│   │   │   │   start_char_idx=5261,
│   │   │   │   end_char_idx=5749,
│   │   │   │   text_template='{metadata_str}\n\n{content}',
│   │   │   │   metadata_template='{key}: {value}',
│   │   │   │   metadata_seperator='\n'
│   │   │   ),
│   │   │   score=0.7966917753219604
│   │   )
│   ],
│   metadata={
│   │   '41ea2d41-b78e-4ff9-8328-81b33c42426d': {
│   │   │   'dl_doc_hash': '556ad9e23b6d2245e36b3208758cf0c8a709382bb4c859eacfe8e73b14e635aa',
│   │   │   'path': '#/main-text/36',
│   │   │   'heading': '3.2 AI models',
│   │   │   'page': 3,
│   │   │   'bbox': [107.25257873535156, 330.1919250488281, 504.05230712890625, 406.1560974121094]
│   │   },
│   │   '5686aae4-1b36-46a4-8f82-e37d23f6d79f': {
│   │   │   'dl_doc_hash': '556ad9e23b6d2245e36b3208758cf0c8a709382bb4c859eacfe8e73b14e635aa',
│   │   │   'path': '#/main-text/10',
│   │   │   'heading': '1 Introduction',
│   │   │   'page': 1,
│   │   │   'bbox': [107.16082763671875, 83.33924865722656, 504.09375, 136.9646759033203]
│   │   }
│   }
)